# Preview Quiz prompt

In [1]:
from pydantic import BaseModel
from openai import OpenAI
import json

client = OpenAI()

In [2]:
prompt = """
[PURPOSE]
Preview lesson content through quizzes to give a sneak peek before the actual lesson.
The quizzes must ask key expressions/the most meaningful words themselves written in GOAL/DESCRIPTION.

[QUIZ GUIDELINES]
- Create two preview quizzes per topic. One from KEY_EXPRESSION, the other from SUB_EXPRESSIONS.
- Put blanks on the KEY_EXPRESSION and SUB_EXPRESSION itself.
- No blanks or tildes on Korean translation
- Blanks must directly ask the key expressions/the most meaningful words themselves.
- Match CEFR level difficulty
- Answer index = the position number of the answer.

[FORMAT]
Each quiz needs:
- A blank (test key patterns itself)
- Complete Korean translation (no blanks or tildes)
- Three options (one correct, two wrong)
- Wrong options must be adequately confusing.
- No possibility of multiple "equally valid" answers
- Shuffle the order of answer choices='answer indexes (1-3)' to avoid predictability.

[Example]
```
Input: 
    {
    "CEFR_level": "A2",
    "DESCRIPTION": "This is my friend, ~와 같은 표현을 사용해 사람을 소개하기.",
    "KEY_EXPRESSION": "A: This is my friend, Jake.",
    "SUB_EXPRESSION": [
      "A: Let me introduce my friend, ~.",
      "A: Meet my friend, ~."
    ]
  }

Output:
    {
    "goal": [
      "다른 사람을 소개하는 표현을 사용할 수 있어요.",
      "This is my friend, ~과 같은 표현을 사용하여 친구를 소개할 수 있어요."
    ],
    "preview_quizzes": [
      {
        "fill_in_blank_quiz": "___ is my friend, Jennie.",
        "korean_translation": "이쪽은 내 친구 Jennie야.",
        "options": ["I", "They", "This"],
        "answer": "This",
        "answer_index": 3
      },
      {
        "fill_in_blank_quiz": "Let me ___ my friend, Sarah.",
        "korean_translation": "내 친구 Sarah를 소개할게.",
        "options": ["introduce", "inter", "intro"],
        "answer": "introduce",
        "answer_index": 2
      }
    ]
  }
}
```
```
[Input]

CEFR level: {{cefr_level}}
DESCRIPTION: {{description}}
KEY EXPRESSION: {{key_expression}}
SUB EXPRESSION: {{sub_expression}}
```

"""

In [ ]:
prompt_template = prompt.format(
    cefr_level = "B1",
    description = "자기소개와 출신지에 대해 말하고, 이에 반응하기.",
    key_expression = "A: My name is Felix. I’m from Australia. B: Glad to meet you.",
    sub_expression = [
        "A: Hi, I’m (). I come from (). B: Nice to meet you.",
        "A: Hello, my name is (). I’m from (). B: Great to meet you."
    ]
)

In [5]:
print(prompt)


[PURPOSE]
Preview lesson content through quizzes to give a sneak peek before the actual lesson.
The quizzes must ask key expressions/the most meaningful words themselves written in GOAL/DESCRIPTION.

[QUIZ GUIDELINES]
- Create two preview quizzes per topic. One from KEY_EXPRESSION, the other from SUB_EXPRESSIONS.
- Put blanks on the KEY_EXPRESSION and SUB_EXPRESSION itself.
- No blanks or tildes on Korean translation
- Blanks must directly ask the key expressions/the most meaningful words themselves.
- Match CEFR level difficulty
- Answer index = the position number of the answer.

[FORMAT]
Each quiz needs:
- A blank (test key patterns itself)
- Complete Korean translation (no blanks or tildes)
- Three options (one correct, two wrong)
- Wrong options must be adequately confusing.
- No possibility of multiple "equally valid" answers
- Shuffle the order of answer choices='answer indexes (1-3)' to avoid predictability.

[Example]
```
Input: 
    {
    "CEFR_level": "A2",
    "DESCRIPTION

In [8]:
class Quiz(BaseModel):
    fill_in_blank_quiz : str
    korean_translation : str
    options : list[str]
    answer : str
    answer_index : int

class PreviewQuiz(BaseModel):
    goal : str
    preview_quizzes : list[Quiz]

In [10]:
completion = client.beta.chat.completions.parse(
    model = 'o3-mini-2025-01-31',
    messages = [
        {"role" : "system", "content" : "You will provide quizzes related to the topics students have learned, appropriately matching their CEFR level for middle school students."},
        {"role" : "user", "content" : prompt}
    ],
    response_format = PreviewQuiz,
)

response = completion.choices[0].message.parsed

In [12]:
response

PreviewQuiz(goal='친근한 인사 표현을 사용할 수 있어요. 그리고 친근하게 인사하는 다양한 표현을 이해할 수 있어요.', preview_quizzes=[Quiz(fill_in_blank_quiz='___, how are you?', korean_translation='안녕, 어떻게 지내?', options=['Hi', 'Hello', 'Hey'], answer='Hello', answer_index=2), Quiz(fill_in_blank_quiz="___, how's it going?", korean_translation='이봐, 어떻게 지내?', options=['Hey', 'Hi', 'Hello'], answer='Hey', answer_index=1)])

In [14]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent = 4)
print(response_output)

{
    "goal": "친근한 인사 표현을 사용할 수 있어요. 그리고 친근하게 인사하는 다양한 표현을 이해할 수 있어요.",
    "preview_quizzes": [
        {
            "fill_in_blank_quiz": "___, how are you?",
            "korean_translation": "안녕, 어떻게 지내?",
            "options": [
                "Hi",
                "Hello",
                "Hey"
            ],
            "answer": "Hello",
            "answer_index": 2
        },
        {
            "fill_in_blank_quiz": "___, how's it going?",
            "korean_translation": "이봐, 어떻게 지내?",
            "options": [
                "Hey",
                "Hi",
                "Hello"
            ],
            "answer": "Hey",
            "answer_index": 1
        }
    ]
}
